In [39]:
import parse

In [252]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2018, day=24)

def parse_line(line):
    fmt = '{units:d} units each with {hp:d} hit points{conditions}with an attack that does {attack:d} {damage} damage at initiative {initiative:d}'
    parsed =  parse.parse(fmt, line).named
    for condition in parsed.pop('conditions').strip('() ').split('; '):
        if condition != '':
            kind, states = condition.split(' to ')
            states = states.split(', ')
            parsed[kind] = tuple(states)
    return parsed

def parses(data):
    armies = data.split('\n\n')
    armies = [[parse_line(line) for line in army.split('\n')[1:]] for army in armies]
    return armies 

data = parses(puzzle.input_data)

In [253]:
sample = parses("""Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4""")

In [353]:
from dataclasses import dataclass
from typing import List, Optional, Tuple
from enum import Enum

@dataclass
class Group:
    units: int
    hp: int
    attack: int
    damage: str
    initiative: int
    idx: int
    army: str = ""
    weak: Tuple[str] = tuple()
    immune: Tuple[str] = tuple()
    
    def __repr__(self):
        return f'{self.army.title()}({self.idx})'
    
    def __hash__(self):
        return hash((self.army, self.idx))
        
    @property
    def power(self):
        return self.units * self.attack
    
    def damage_to(self, other):
        if self.army == other.army:
            return 0
        if self.damage in other.immune:
            return 0
        dmg = self.power 
        if self.damage in other.weak:
            dmg *= 2
        return dmg
    
    def selection_order(self):
        return (self.power, self.initiative)
    
    def target_order(self, other):
        return (self.damage_to(other), other.power, other.initiative)
    
    def attack_order(self):
        return self.initiative
    
    def deal_damage(self, other):
        if self.units > 0 and other.units > 0:
            dmg = self.damage_to(other)
            other.units = max(0, other.units-dmg//other.hp)
        

In [354]:
def combat(data, immune_boost=0):
    army1, army2 = data
    immune = [Group(**vals,army='immune', idx=i+1) for i, vals in enumerate(army1)]
    infection = [Group(**vals,army='infection', idx=i+1) for i, vals in enumerate(army2)]
    
    for grp in immune:
        grp.attack += immune_boost
    
    while len(immune) != 0 and len(infection) != 0:
        before_units = [grp.units for grp in immune+infection]
        # Target selection
        targeting = {}
        targeted = set()
        enemies = {'immune': infection, 'infection': immune}
    
        for grp in sorted(infection+immune, reverse=True, key=Group.selection_order):
            targets = [x for x in enemies[grp.army] if x not in targeted]
            if targets:
                target = max(targets, key=grp.target_order)
                if (dmg := grp.damage_to(target)) > 0:
                    targeting[grp] = target
                    targeted.add(target)

        # Combat
        for grp in sorted(infection+immune, reverse=True, key=Group.attack_order):
            if (target := targeting.get(grp, None)):
                before = target.units
                grp.deal_damage(target)

        # Remove dead units
        immune = [grp for grp in immune if grp.units > 0]
        infection = [grp for grp in infection if grp.units > 0]
        
        # Check stalemate
        after_units = [grp.units for grp in immune+infection]
        if before_units == after_units:
            break
                
    return immune, infection

In [355]:
def solve_a(data):
    immune, infection = data
    return sum(grp.units for grp in immune+infection)

In [356]:
def solve_b(data):

    for boost in itertools.count(1):
        print(boost)
        immune, infection = combat(data, boost)
        if len(immune) > 0 and len(infection) == 0:
            break
    print(boost)
    return sum(grp.units for grp in immune)

In [350]:
def solve_b(data):
    left, right = 0, 10_000
    while right-left > 1:
        mid = (left+right)//2
        immune, infection = combat(data, mid)
        if len(infection) > 0:
            left = mid
        else:
            right = mid
    immune, infection = combat(data, right)
    return sum(grp.units for grp in immune)

In [351]:
solve_b(sample)

51

In [352]:
solve_b(data)

862

In [342]:
combat(data, immune_boost=46)

([Immune(6), Immune(10)], [Infection(2)])

In [230]:
immune, infection

([Immune(2)], [Infection(1), Infection(2)])

In [194]:
for grp in infection+immune:
    print(grp)

Group(units=797, hp=4706, attack=116, damage='bludgeoning', initiative=1, idx=1, army='infection', weak=('radiation',), immune=())
Group(units=4485, hp=2961, attack=12, damage='slashing', initiative=4, idx=2, army='infection', weak=('fire', 'cold'), immune=('radiation',))
Group(units=0, hp=5390, attack=4507, damage='fire', initiative=2, idx=1, army='immune', weak=('radiation', 'bludgeoning'), immune=())
Group(units=905, hp=1274, attack=25, damage='slashing', initiative=3, idx=2, army='immune', weak=('bludgeoning', 'slashing'), immune=('fire',))


In [195]:
def target_selection(groups):
    targeting = {}
    targeted = {}
    
    for grp in sorted(groups, reverse=True, key=Group.target_order):
        print(grp)
        

In [79]:
sorted([True, False, True], reverse=True)

[True, True, False]

In [54]:
sample

[[{'units': 17,
   'hp': 5390,
   'damage': 4507,
   'dmg_type': 'fire',
   'initiative': 2,
   'weak': ['radiation', 'bludgeoning']},
  {'units': 989,
   'hp': 1274,
   'damage': 25,
   'dmg_type': 'slashing',
   'initiative': 3,
   'immune': ['fire'],
   'weak': ['bludgeoning', 'slashing']}],
 [{'units': 801,
   'hp': 4706,
   'damage': 116,
   'dmg_type': 'bludgeoning',
   'initiative': 1,
   'weak': ['radiation']},
  {'units': 4485,
   'hp': 2961,
   'damage': 12,
   'dmg_type': 'slashing',
   'initiative': 4,
   'immune': ['radiation'],
   'weak': ['fire', 'cold']}]]

In [36]:
conditions = _['conditions']
cs = {}


In [37]:
cs

{'weak': ['radiation', 'bludgeoning']}